In [16]:
import pandas as pd
import requests
import numpy as np
import requests
import bs4
import re
from datetime import datetime
import csv
import os

In [17]:
movies= pd.read_csv('IMDB movies.csv', low_memory=False)
currency= pd.read_csv('Currency.csv', low_memory=False)

In [18]:
budgetmovies = movies[movies['budget'].notnull()].copy()
budgetmovies['currency'] = budgetmovies.loc[:,'budget'].str.slice(0,3)
budgetmovies['currency'] = budgetmovies['currency'].apply(lambda x: 'USD' if '$' in x else x)
budgetmovies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,currency
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0,USD
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0,USD
7,tt0002445,Quo Vadis?,Quo Vadis?,1913,1913-03-01,"Drama, History",120,Italy,Italian,Enrico Guazzoni,...,"An epic Italian film ""Quo Vadis"" influenced ma...",6.2,273,ITL 45000,NaN,NaN,NaN,7.0,5.0,ITL
8,tt0002452,Independenta Romaniei,Independenta Romaniei,1912,1912-09-01,"History, War",120,Romania,NaN,"Aristide Demetriade, Grigore Brezeanu",...,The movie depicts the Romanian War of Independ...,6.7,198,ROL 400000,NaN,NaN,NaN,4.0,1.0,ROL
9,tt0002461,Richard III,Richard III,1912,1912-10-15,Drama,55,"France, USA",English,"André Calmettes, James Keane",...,Richard of Gloucester uses manipulation and mu...,5.5,225,$ 30000,NaN,NaN,NaN,8.0,1.0,USD


In [19]:
print(budgetmovies["worlwide_gross_income"].isna().sum().sum())
print(budgetmovies["usa_gross_income"].isna().sum().sum())
usamovies = budgetmovies[budgetmovies["usa_gross_income"].notnull()].copy()
print(len(usamovies))

10948
15584
8126


In [27]:
usamovies1 = usamovies[usamovies["country"].notnull()].copy()
usamovies2 = usamovies1[usamovies1["country"].str.match("USA")]
usamovies3 = usamovies[usamovies["currency"].str.match("USD")]
print(len(usamovies3))
print(len(usamovies2))


7282
5571


In [30]:
usamovies2.to_csv('USAMOVIES.csv')

In [ ]:
#possible way to merge but will have to go line by line
NYUSCounties=pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

NYALCounties=NYUSCounties[NYUSCounties['state']=="Alabama"]
NYALCounties = NYALCounties.sort_values(['county', 'date'])
NYALCounties=NYALCounties.reset_index(drop=True)
NYALCounties['date'] = pd.to_datetime(NYALCounties['date'])
NYALCounties['date'] = NYALCounties['date'].dt.date

#Pivoting to calculated diff and fill na-- for Daily Cases
sorted_data = NYALCounties.pivot(index = 'date', columns = 'county', values = 'cases')
sorted_data = sorted_data.fillna(0)
for col in sorted_data.columns:
    sorted_data[col] = sorted_data[col].diff()
sorted_data = sorted_data.reset_index()
#reformating data to be able to merge
melted_data = pd.melt(sorted_data, id_vars = 'date')
melted_data = melted_data.fillna(0)
melted_data = melted_data.rename(columns = {'value': 'Daily_Cases'})
#merging Daily Cases into dataframe
NYALCounties = NYALCounties.merge(melted_data, how = 'left')